## <span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#37ABC4;"> Mobile Recommendation System

In [ ]:
#Importing all the libraries
import pandas as pd

#Machine learning libraries(Supervised Learning)
from sklearn.model_selection import train_test_split

from surprise import SVD, KNNWithMeans, Dataset, accuracy, Reader
from surprise.model_selection import cross_validate, train_test_split
from collections import defaultdict

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [2]:
# Merging all CSVs into one DataFrame
def read_csv1(csv_or_excel_path):
    if csv_or_excel_path[-3:]=='csv':
        try: pd.read_csv(csv_or_excel_path)
        except:
            try: 
                return pd.read_csv(csv_or_excel_path,sep=',')
            except:
                try: 
                    return pd.read_csv(csv_or_excel_path,sep='\t')
                except:
                    try: 
                        return pd.read_csv(csv_or_excel_path,encoding='utf-8')
                    except:
                        try: 
                            return pd.read_csv(csv_or_excel_path,encoding='utf-8',sep=',')
                        except:
                            try: 
                                return pd.read_csv(csv_or_excel_path,encoding='utf-8',sep='\t')
                            except:
                                try: 
                                    return pd.read_csv(csv_or_excel_path,encoding = "ISO-8859-1", sep=",")
                                except:
                                    try: 
                                        return pd.read_csv(csv_or_excel_path,encoding = "ISO-8859-1", sep=",")
                                    except: 
                                        return pd.read_csv(csv_or_excel_path,encoding = "ISO-8859-1", sep="\t")

In [3]:
data1 = read_csv1("C:/Users/Lenovo/Documents/Data Set/phone_user_review_file_1.csv")

In [4]:
data1.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [5]:
data1.shape

(374910, 11)

In [6]:
data2 = read_csv1("C:/Users/Lenovo/Documents/Data Set/phone_user_review_file_2.csv")
data2.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/leagoo-lead-7/,4/15/2015,en,us,Amazon,amazon.com,2.0,10.0,"The telephone headset is of poor quality , not...",luis,Leagoo Lead7 5.0 Inch HD JDI LTPS Screen 3G Sm...
1,/cellphones/leagoo-lead-7/,5/23/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,This is my first smartphone so I have nothing ...,Mark Lavin,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
2,/cellphones/leagoo-lead-7/,4/27/2015,en,gb,Amazon,amazon.co.uk,8.0,10.0,Great phone. Battery life not great but seems ...,tracey,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
3,/cellphones/leagoo-lead-7/,4/22/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,Best 90 quid I've ever spent on a smart phone,Reuben Ingram,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
4,/cellphones/leagoo-lead-7/,4/18/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,I m happy with this phone.it s very good.thx team,viorel,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...


In [7]:
data2.shape

(114925, 11)

In [8]:
data3 = read_csv1("C:/Users/Lenovo/Documents/Data Set/phone_user_review_file_3.csv")
data3.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,11/7/2015,pt,br,Submarino,submarino.com.br,6.0,10.0,"recomendo, eu comprei um, a um ano, e agora co...",herlington tesch,Samsung Smartphone Samsung Galaxy S3 Slim G381...
1,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,10/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,Comprei um pouco desconfiada do site e do celu...,Luisa Silva Marieta,Samsung Smartphone Samsung Galaxy S3 Slim G381...
2,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,"Muito bom o produto, obvio que tem versÃµes me...",Cyrus,Samsung Smartphone Samsung Galaxy S3 Slim G381...
3,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,8.0,10.0,Unica ressalva fica para a camera que poderia ...,Marcela Santa Clara Brito,Samsung Smartphone Samsung Galaxy S3 Slim G381...
4,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/1/2015,pt,br,Colombo,colombo.com.br,8.0,10.0,Rapidez e atenÃ§Ã£o na entrega. O aparelho Ã© ...,Claudine Maria Kuhn Walendorff,"Smartphone Samsung Galaxy S3 Slim, Dual Chip, ..."


In [9]:
data3.shape

(312961, 11)

In [10]:
data4 = read_csv1("C:/Users/Lenovo/Documents/Data Set/phone_user_review_file_4.csv")
data4.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-s7262-duos-galaxy-ace/,3/11/2015,en,us,Amazon,amazon.com,2.0,10.0,was not conpatable with my phone as stated. I ...,Frances DeSimone,Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce...
1,/cellphones/samsung-s7262-duos-galaxy-ace/,17/11/2015,en,in,Zopper,zopper.com,10.0,10.0,Decent Functions and Easy to Operate Pros:- Th...,Expert Review,Samsung Galaxy Star Pro S7262 Black
2,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,4.0,10.0,Not Good Phone such price. Hang too much and v...,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
3,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,6.0,10.0,not bad for features,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
4,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,10.0,10.0,Excellent product,NHK,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)


In [11]:
data4.shape

(98284, 11)

In [12]:
data5 = read_csv1("C:/Users/Lenovo/Documents/Data Set/phone_user_review_file_5.csv")
data5.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2.0,10.0,I bought 1 month before. currently speaker is ...,venkatesh,Karbonn K1616
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6.0,10.0,"I just bought one week back, I have Airtel con...",Venkat,Karbonn K1616
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4.0,10.0,one problem in this handset opera is not worki...,krrish,Karbonn K1616
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10.0,10.0,here Karbonn comes up with an another excellen...,BRIJESH CHAUHAN,Karbonn K1616 - Black
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10.0,10.0,"What a phone, all so on Naaptol my god 23% off...",Suraj CHAUHAN,Karbonn K1616 - Black


In [13]:
data5.shape

(350216, 11)

In [14]:
data6 = read_csv1("C:/Users/Lenovo/Documents/Data Set/phone_user_review_file_6.csv")
data6.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-instinct-sph-m800/,9/16/2011,en,us,Phone Arena,phonearena.com,8.0,10.0,I've had the phone for awhile and it's a prett...,ajabrams95,Samsung Instinct HD
1,/cellphones/samsung-instinct-sph-m800/,2/13/2014,en,us,Amazon,amazon.com,6.0,10.0,to be clear it is not the sellers fault that t...,Stephanie,Samsung SPH M800 Instinct
2,/cellphones/samsung-instinct-sph-m800/,12/30/2011,en,us,Phone Scoop,phonescoop.com,9.0,10.0,Well i love this phone. i have had ton of phon...,snickers,Instinct M800
3,/cellphones/samsung-instinct-sph-m800/,10/18/2008,en,us,HandCellPhone,handcellphone.com,4.0,10.0,I have had my Instinct for several months now ...,A4C,Samsung Instinct
4,/cellphones/samsung-instinct-sph-m800/,9/6/2008,en,us,Reviewed.com,reviewed.com,6.0,10.0,i have had this instinct phone for about two m...,betaBgood,Samsung Instinct


In [15]:
data6.shape

(163837, 11)

In [16]:
combined = pd.concat([data1, data2], ignore_index=True)
combined.shape

(489835, 11)

In [17]:
df = pd.concat([combined, data3], ignore_index=True)
df.shape

(802796, 11)

In [18]:
df = pd.concat([df, data4], ignore_index=True)
df.shape

(901080, 11)

In [19]:
df = pd.concat([df, data5], ignore_index=True)
df.shape

(1251296, 11)

In [20]:
df = pd.concat([df, data3], ignore_index=True)
df.shape

(1564257, 11)

In [21]:
#Combined CSV dataframe
df

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
1564252,/cellphones/samsung-galaxy-s-iii/,1/23/2014,de,de,Yopi,yopi.de,10.0,10.0,Ein Disney- Meisterwerk fÃ¼r jung und alt! Zei...,burnson,Samsung Galaxy Express I8730
1564253,/cellphones/samsung-galaxy-s-iii/,1/23/2014,de,de,Yopi,yopi.de,NaN,NaN,Nachdem ich mir einen neuen 3D fÃ¤higen Fernse...,Comflag,Samsung Galaxy Express I8730
1564254,/cellphones/samsung-galaxy-s-iii/,1/22/2014,ru,ru,Yandex,market.yandex.ru,8.0,10.0,ÐÐ¾ Ð¿Ð¾Ð²Ð¾Ð´Ñ Ð°ÐºÐºÑÐ¼ÑÐ»ÑÑÐ¾ÑÐ°. Ð...,NaN,Samsung Galaxy S III GT-I9300 16Gb
1564255,/cellphones/samsung-galaxy-s-iii/,1/22/2014,pt,br,Submarino,submarino.com.br,10.0,10.0,Ã³timo primeira linha acho que todo mundo gost...,magda vanessa ferreira,Samsung Smartphone Samsung Galaxy S III I9300 ...


### <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#D70D07;"> Data Understanding & Exploration:

In [22]:
df.describe()

,score,score_max
count,1.504412e+06,1504412.0
mean,8.022120e+00,10.0
std,2.647502e+00,0.0
min,2.000000e-01,10.0
25%,7.700000e+00,10.0
50%,9.900000e+00,10.0
75%,1.000000e+01,10.0
max,1.000000e+01,10.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564257 entries, 0 to 1564256
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1564257 non-null  object 
 1   date       1564257 non-null  object 
 2   lang       1564257 non-null  object 
 3   country    1564257 non-null  object 
 4   source     1564257 non-null  object 
 5   domain     1564257 non-null  object 
 6   score      1504412 non-null  float64
 7   score_max  1504412 non-null  float64
 8   extract    1545054 non-null  object 
 9   author     1503324 non-null  object 
 10  product    1564255 non-null  object 
dtypes: float64(2), object(9)
memory usage: 131.3+ MB


In [24]:
nullPercent = ((df.isna().sum()/1564257 )*100).round(2)
nullPercent

phone_url    0.00
date         0.00
lang         0.00
country      0.00
source       0.00
domain       0.00
score        3.83
score_max    3.83
extract      1.23
author       3.90
product      0.00
dtype: float64

#### Observations:
1. The columns score, score_max, extract and author has null values
2. Only score and score_max datatype is float, others are object
3. The datatype of the column "date" should be in datetype.

In [25]:
# Round off scores to the nearest integers
df['score'] = df['score'].round(0).astype('Int64')
print(' Round-off scores : \n',list(df.score.unique()))

 Round-off scores : 
 [10, 6, 9, 4, 8, 2, 7, 5, <NA>, 3, 1, 0]


In [26]:
df.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        59845
score_max    59845
extract      19203
author       60933
product          2
dtype: int64

In [27]:
df['score'] = df['score'].fillna(df['score'].median())
df['score_max'] = df['score_max'].fillna(df['score_max'].median())
print('Imputed values: \n',list(df.score.unique()))
print('Imputed values: \n',list(df.score_max.unique()))

Imputed values: 
 [10, 6, 9, 4, 8, 2, 7, 5, 3, 1, 0]
Imputed values: 
 [10.0]


In [28]:
df[df.duplicated()]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
342,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,10,10.0,Ik vind het tot nu toe een heel erg aangenaam ...,Stefanie Hinneman,Samsung Galaxy S8 Zwart
343,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,10,10.0,Ik vind het tot nu toe een heel erg aangenaam ...,Stefanie Hinneman,Samsung Galaxy S8 Zwart
371,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,10,10.0,Tot nu toe in de eerste dagen weinig tot geen ...,Robin Winckens,Samsung Galaxy S8 Zwart
452,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,10,10.0,Wat een huge scherm!,B Yapar,Samsung Galaxy S8 Zwart
3718,/cellphones/samsung-galaxy-s6-edgeplus/,10/4/2015,en,us,Amazon,amazon.com,10,10.0,"For readers, my opinions on this device (ATT 3...",FrozenCloud,"Samsung Galaxy S6 Edge+, Black 64GB (Verizon W..."
...,...,...,...,...,...,...,...,...,...,...,...
1564252,/cellphones/samsung-galaxy-s-iii/,1/23/2014,de,de,Yopi,yopi.de,10,10.0,Ein Disney- Meisterwerk fÃ¼r jung und alt! Zei...,burnson,Samsung Galaxy Express I8730
1564253,/cellphones/samsung-galaxy-s-iii/,1/23/2014,de,de,Yopi,yopi.de,10,10.0,Nachdem ich mir einen neuen 3D fÃ¤higen Fernse...,Comflag,Samsung Galaxy Express I8730
1564254,/cellphones/samsung-galaxy-s-iii/,1/22/2014,ru,ru,Yandex,market.yandex.ru,8,10.0,ÐÐ¾ Ð¿Ð¾Ð²Ð¾Ð´Ñ Ð°ÐºÐºÑÐ¼ÑÐ»ÑÑÐ¾ÑÐ°. Ð...,NaN,Samsung Galaxy S III GT-I9300 16Gb
1564255,/cellphones/samsung-galaxy-s-iii/,1/22/2014,pt,br,Submarino,submarino.com.br,10,10.0,Ã³timo primeira linha acho que todo mundo gost...,magda vanessa ferreira,Samsung Smartphone Samsung Galaxy S III I9300 ...


In [29]:
df = df.drop_duplicates()
df

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
1251291,/cellphones/sony-ericsson-z710i/,8/7/2006,fr,fr,GraphMobile,graphmobile.com,10,10.0,Pour info il est sur amazon.de a 212.99â¬ s'i...,NaN,Sony-Ericsson Z710i
1251292,/cellphones/sony-ericsson-z710i/,8/5/2006,fr,fr,GraphMobile,graphmobile.com,9,10.0,HabituÃ© Ã samsung sony nous sort 1 jolie cla...,NaN,Sony-Ericsson Z710i
1251293,/cellphones/sony-ericsson-z710i/,7/19/2006,fr,fr,GraphMobile,graphmobile.com,10,10.0,"Pour les gens qui ne regarde pas Il fait mp3, ...",NaN,Sony-Ericsson Z710i
1251294,/cellphones/sony-ericsson-z710i/,7/9/2006,fr,fr,GraphMobile,graphmobile.com,9,10.0,C vrai que sans le mp3 c moyen...,NaN,Sony-Ericsson Z710i


In [30]:
# Keeping only 1 million data samples with random state of 612
df = df.sample(n=1000000, random_state=612)
print(df.shape)
df

(1000000, 11)


,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
527108,/cellphones/kyocera-duraxv-duraxa/,5/16/2015,en,us,Amazon,amazon.com,10,10.0,This item Kyocera DuraXV was is exactly as adv...,darryl,"Kyocera DuraXV, Non Camera Dark Grey 4GB (Veri..."
167101,/cellphones/microsoft-lumia-640-dual-sim/,2/13/2016,es,es,Amazon,amazon.es,10,10.0,Cuando lo recibÃ­ lo primero que pensÃ© fue qu...,Amazon Customer,Microsoft Lumia 640 - Smartphone libre Windows...
1237276,/cellphones/motorola-i776/,4/17/2009,pt,br,MercadoLibre,opiniao.mercadolivre.com.br,10,10.0,Aparelho muito excelente pelo custo benefico,JOSECDSANTOS2006,Motorola i776
53446,/cellphones/apple-iphone-7/,3/1/2017,de,de,Otto.de,otto.de,6,10.0,Das iPhone 7 ist fÃ¼r meinen persÃ¶nlichen Ges...,einem Kunden,"Apple iPhone 7 4,7"" 256 GB"
225935,/cellphones/motorola-moto-g3/,12/20/2016,it,it,Amazon,amazon.it,10,10.0,"Acquistato circa un anno fa, uno smartphone ve...",Mazznico,Lenovo Motorola Moto G XT1541 3a Generazione S...
...,...,...,...,...,...,...,...,...,...,...,...
161697,/cellphones/samsung-galaxy-a3-2016/,12/14/2016,it,it,Amazon,amazon.it,10,10.0,"cercavo un cellulare di queste dimensioni, e i...",Andrea Dini,"Samsung Galaxy A3 (2016) Smartphone, 4.7"", 16 ..."
607959,/cellphones/karbonn-sparkle-v/,1/3/2015,en,in,Amazon,amazon.in,10,10.0,Very good phone for beginners.,BRIJESH,Karbonn Sparkle V (Red)
438876,/cellphones/lg-g2-594708/,4/13/2015,it,it,Pixmania,pixmania.it,10,10.0,Il prodotto anche se Ã¨ del 2014 a inizio 2015...,bizkit,LG G2 32 GB - nero - Smartphone
619041,/cellphones/htc-desire-610/,12/16/2016,en,us,Wireless AT&T,att.com,2,10.0,Absolutely horrible phone. I have to put it on...,Kurn,HTC Desire 610 (AT&amp;T Certified Pre-Owned)


In [31]:
# Dropping irrelevant features
df.drop(['phone_url', 'date','lang','country','domain','score_max','extract'], axis=1,inplace=True)
df

,source,score,author,product
527108,Amazon,10,darryl,"Kyocera DuraXV, Non Camera Dark Grey 4GB (Veri..."
167101,Amazon,10,Amazon Customer,Microsoft Lumia 640 - Smartphone libre Windows...
1237276,MercadoLibre,10,JOSECDSANTOS2006,Motorola i776
53446,Otto.de,6,einem Kunden,"Apple iPhone 7 4,7"" 256 GB"
225935,Amazon,10,Mazznico,Lenovo Motorola Moto G XT1541 3a Generazione S...
...,...,...,...,...
161697,Amazon,10,Andrea Dini,"Samsung Galaxy A3 (2016) Smartphone, 4.7"", 16 ..."
607959,Amazon,10,BRIJESH,Karbonn Sparkle V (Red)
438876,Pixmania,10,bizkit,LG G2 32 GB - nero - Smartphone
619041,Wireless AT&T,2,Kurn,HTC Desire 610 (AT&amp;T Certified Pre-Owned)


In [32]:
# The most rated products
print('Top 10 rated products:\n\n',df['product'].value_counts().head(10))

Top 10 rated products:

 Lenovo Vibe K4 Note (White,16GB)       4169
Lenovo Vibe K4 Note (Black, 16GB)      3512
OnePlus 3 (Graphite, 64 GB)            3267
OnePlus 3 (Soft Gold, 64 GB)           2853
Samsung Galaxy Express I8730           2175
Huawei P8lite zwart / 16 GB            2129
Lenovo Vibe K5 (Gold, VoLTE update)    2022
Samsung Galaxy S6 zwart / 32 GB        1889
Nokia 5800 XpressMusic                 1701
Lenovo Vibe K5 (Grey, VoLTE update)    1701
Name: product, dtype: int64


In [33]:
# Users with most number of reviews
print('Users with most number of reviews:\n\n',df['author'].value_counts().head())

Users with most number of reviews:

 Amazon Customer    60872
Cliente Amazon     15460
e-bit               6735
Client d'Amazon     6154
Amazon Kunde        3717
Name: author, dtype: int64


In [34]:
# Selecting the data with products having more than 50 ratings and users who have given more than 50 ratings
user_50= df['author'].value_counts()
user_50

Amazon Customer                 60872
Cliente Amazon                  15460
e-bit                            6735
Client d'Amazon                  6154
Amazon Kunde                     3717
                                ...  
Rajiv Behl                          1
Ana Paula Pereira Tang Vidal        1
ToyoWilly                           1
andrey-ratkov                       1
CARIMEDA                            1
Name: author, Length: 583849, dtype: int64

In [36]:
user_50 = user_50[user_50>50].index.tolist()
user_50

['Amazon Customer',
 'Cliente Amazon',
 'e-bit',
 "Client d'Amazon",
 'Amazon Kunde',
 'einer Kundin',
 'Anonymous',
 'einem Kunden',
 'unknown',
 'Anonymous ',
 'Ð\x90Ð»ÐµÐºÑ\x81Ð°Ð½Ð´Ñ\x80',
 'David',
 'Ð¡ÐµÑ\x80Ð³ÐµÐ¹',
 'Marco',
 'Ð\x94Ð¼Ð¸Ñ\x82Ñ\x80Ð¸Ð¹',
 'Andrea',
 'Alex',
 'Ð\x90Ð½Ð´Ñ\x80ÐµÐ¹',
 'Ð\x90Ð»ÐµÐºÑ\x81ÐµÐ¹',
 'ZÃ¡kaznÃ\xadk',
 'Daniel',
 'Antonio',
 'Chris',
 'Francesco',
 'Michael',
 'Alessandro',
 'Kindle Customer',
 'Roberto',
 'Misafir',
 'John',
 'Luca',
 'AnÃ´nimo',
 'Peter',
 'Amazon-Kunde',
 'Alberto',
 'Ð\x95Ð²Ð³ÐµÐ½Ð¸Ð¹',
 'Ð\x93Ð¾Ñ\x81Ñ\x82Ñ\x8c',
 'Carlos',
 'Ð\x92Ð»Ð°Ð´Ð¸Ð¼Ð¸Ñ\x80',
 'Mike',
 'Fabio',
 'Stefano',
 'Anonyme',
 'Giuseppe',
 'Ð\x94ÐµÐ½Ð¸Ñ\x81',
 'Paul',
 'Jose',
 'anonieme bezoeker',
 'Martin',
 '??????????',
 'Javier',
 'Laura',
 'Christian',
 'Thomas',
 'Ð\x98Ð³Ð¾Ñ\x80Ñ\x8c',
 'Manuel',
 'Ð\x9fÐ¾Ð»Ñ\x8cÐ·Ð¾Ð²Ð°Ñ\x82ÐµÐ»Ñ\x8c Ñ\x81ÐºÑ\x80Ñ\x8bÐ» Ñ\x81Ð²Ð¾Ð¸ Ð´Ð°Ð½Ð½Ñ\x8bÐµ',
 'Davide',
 'Frank',
 'Sergio',
 'Paolo',
 'Ð\x9eÐ»Ñ\x8cÐ³Ð°',
 '

In [38]:
product_50 = df['product'].value_counts()
product_50 = product_50[product_50>50].index.tolist()
product_50

['Lenovo Vibe K4 Note (White,16GB)',
 'Lenovo Vibe K4 Note (Black, 16GB)',
 'OnePlus 3 (Graphite, 64 GB)',
 'OnePlus 3 (Soft Gold, 64 GB)',
 'Samsung Galaxy Express I8730',
 'Huawei P8lite zwart / 16 GB',
 'Lenovo Vibe K5 (Gold, VoLTE update)',
 'Samsung Galaxy S6 zwart / 32 GB',
 'Nokia 5800 XpressMusic',
 'Lenovo Vibe K5 (Grey, VoLTE update)',
 'Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)',
 'OnePlus 3T (Gunmetal, 6GB RAM + 64GB memory)',
 'Samsung Galaxy J3 (8GB)',
 'Samsung Galaxy S7 edge 32GB (Verizon)',
 'Nokia N8',
 'Samsung Galaxy S7 edge 32GB (T-Mobile)',
 "Huawei P9 Lite Smartphone, LTE, Display 5.2'' FHD, Processore Octa-Core Kirin 650, 16 GB Memoria Interna, 3GB RAM, Fotocamera 13 MP, Single-SIM, Android 6.0 Marshmallow, Bianco [Italia]",
 'Samsung Galaxy S6 edge zwart / 32 GB',
 'Samsung Galaxy S7 32GB (Verizon)',
 'Apple iPhone 5s (Silver, 16GB)',
 'Samsung Galaxy S7 edge Smartphone, 13,9 cm (5,5 Zoll) Display, LTE (4G)',
 'Nokia 5800',
 'YU Yuphoria YU5010A (Black+Silve

In [39]:
review_50 = df[(df['author'].isin(user_50)) & (df['product'].isin(product_50))]
print("Shape of the dataset:\n",review_50.shape)
review_50.head()

Shape of the dataset:
 (118162, 4)


,source,score,author,product
167101,Amazon,10,Amazon Customer,Microsoft Lumia 640 - Smartphone libre Windows...
53446,Otto.de,6,einem Kunden,"Apple iPhone 7 4,7"" 256 GB"
344701,Flipkart,10,Samuel,"SAMSUNG Galaxy S5 (Copper Gold, 16 GB)"
113897,KIESKEURIG,8,Robert,Huawei P8 grijs / 16 GB
1146572,Schede-Cellulari.it,10,simone,BlackBerry Curve 8900


### <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#D70D07;">Building a popularity based model and recommend top 5 mobile phones

In [40]:
def popularity_based(data):
    ratings_mean_count = pd.DataFrame(data.groupby('product')['score'].mean())
    ratings_mean_count['rating_counts'] = data.groupby('product')['score'].count()
    ratings_mean_count = ratings_mean_count.sort_values(by=['score','rating_counts'], ascending=[False,False])
    print('Top 5 recommendations for the products are: \n')
    display(ratings_mean_count.head())
    return

In [41]:
popularity_based(df)

Top 5 recommendations for the products are: 



,score,rating_counts
product,,
SAMSUNG Player Star 2,10.0,520
SAMSUNG Galaxy S,10.0,327
NOKIA E71 Noir,10.0,287
Vodafone (69) 858 Smart,10.0,254
Nokia 5800 XpressMusic Cep Telefonu,10.0,253


### <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#D70D07;"> Building a collaborative filtering model using kNNWithMeans from surprise.

In [47]:
#Rearranging the columns for SVD
df1 = Dataset.load_from_df(df[['author','product','score']], Reader(rating_scale=(1, 10)))

In [48]:
# splitting train and test sets
trainset, testset = train_test_split(df1, test_size=0.25,random_state=612)

In [50]:
svd = SVD(random_state=612)
svd.fit(trainset)
svd_pred = svd.test(testset)
print('First few prediction values: \n',svd_pred[0:2])

First few prediction values: 
 [Prediction(uid='JeeJeeJerryCotton666', iid='Motorola Moto G 5th gen. (2017) Dual-SIM â\x80\x90Android-puhelin, harmaa', r_ui=8.0, est=8.35831660173878, details={'was_impossible': False}), Prediction(uid='KCNA', iid='Microsoft Lumia 535 (Black, 8GB)', r_ui=10.0, est=6.890146616751126, details={'was_impossible': False})]


In [57]:
# Reducing Sample size fro computation efficiency 
df_reduced= df.sample(n=5000, random_state=612)
print(df_reduced.shape)
df_reduced

(5000, 4)


,source,score,author,product
302837,Amazon,2,NIL KANTH SINGH,"Asus ZenFone 2 Unlocked Cellphone, 64GB, Red (..."
564737,Amazon,10,Elena Hambitzer,"Samsung Galaxy Note 3 Smartphone (14,5 cm (5,7..."
858883,SmartphoneShop,9,Willem,Acer CloudMobile
471797,Amazon,10,Amazon Customer,"Asus ZenFone 5 Smartphone, 8 GB, RAM 2 GB, Ner..."
1240275,Hotline.ua,10,NaN,LG KP152
...,...,...,...,...
386157,Amazon,8,Praveen kumar,"Motorola Moto E 2nd Generation (3G, White)"
953839,Yandex,10,ÐÐ¸ÐºÐ¸ÑÐ¾ÑÐ¾Ð²Ð° ÐÐ°ÑÐ°Ð»ÑÑÂ ÐÐ½Ð°ÑÐ...,Sony Xperia tipo
688190,Amazon,10,Amazon Customer,"Samsung Galaxy S4 GT-I9500 (White Frost, 16GB)"
57519,Amazon,8,Sujith,"Apple iPhone SE (Space Grey, 64GB)"


In [58]:
df2 = Dataset.load_from_df(df_reduced[['author','product','score']], Reader(rating_scale=(1, 10)))

In [61]:
trainset2, testset2 = train_test_split(df2, test_size=0.25,random_state=612)

In [62]:
# Collaborative filtering model using kNNWithMeans_Item based
knn_i = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
knn_i.fit(trainset2)
knn_i_pred = knn_i.test(testset2)
print('First few prediction values: \n',knn_i_pred[0:2])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
First few prediction values: 
 [Prediction(uid='Al-Man', iid='Lenovo Moto Z Play Smartphone (14 cm (5,5 Zoll), 32 GB, Android) [Spanische Version] Schwarz/Silber', r_ui=10.0, est=8.158666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}), Prediction(uid='Bubbles', iid='Nokia Asha 300 Sim Free Mobile Phone - Graphite', r_ui=10.0, est=8.158666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]


In [64]:
# Collaborative filtering model using kNNWithMeans_User based
knn_u = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
knn_u.fit(trainset2)
knn_u_pred = knn_u.test(testset2)
print('First few prediction values: \n',knn_u_pred[0:2])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
First few prediction values: 
 [Prediction(uid='Al-Man', iid='Lenovo Moto Z Play Smartphone (14 cm (5,5 Zoll), 32 GB, Android) [Spanische Version] Schwarz/Silber', r_ui=10.0, est=8.158666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}), Prediction(uid='Bubbles', iid='Nokia Asha 300 Sim Free Mobile Phone - Graphite', r_ui=10.0, est=8.158666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]


### <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#D70D07;"> Evaluating the collaborative models

In [69]:
# RMSE values for all the 3 models
print('SVD- RMSE value(test-set):',round(accuracy.rmse(svd_pred),2)) 
print('Knn_i-RMSE value(Item-based Model, test-set): ',round(accuracy.rmse(knn_i_pred),2))
print('Knn_u-RMSE value(Item-based Model, test-set): ',round(accuracy.rmse(knn_u_pred),2))

RMSE: 2.4912
SVD- RMSE value(test-set): 2.49
RMSE: 2.5966
Knn_i-RMSE value(Item-based Model, test-set):  2.6
RMSE: 2.6081
Knn_u-RMSE value(Item-based Model, test-set):  2.61


In [70]:
# Predicting score (average rating) for test users
svd_pred_df=pd.DataFrame(svd_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
print('average rating  by test users(SVD): ',svd_pred_df['rui'].mean())
Knn_i_pred_df=pd.DataFrame(knn_i_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
print('average rating  by test users(KNN_itemBased): ',Knn_i_pred_df['rui'].mean())
Knn_u_pred_df=pd.DataFrame(knn_u_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
print('average rating  by test users(KNN_userBased): ',Knn_u_pred_df['rui'].mean())

average rating  by test users(SVD):  8.130396
average rating  by test users(KNN_itemBased):  8.164
average rating  by test users(KNN_userBased):  8.164


1. The mobile phone which is rated 10 by highest number of people is SAMSUNG Player Star 2
2. The RMSE for SVD, KNN item based and KNN user based as similar

In [73]:
# Recommending top 5 products for the test users

def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
     
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the n highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_5 = get_top_n(knn_i_pred,5)
print('Top 5 recommendations for all test users are: \n')
for key,value in top_5.items(): print(key,'-> ',value,'\n')

Top 5 recommendations for all test users are: 

Al-Man ->  [('Lenovo Moto Z Play Smartphone (14 cm (5,5 Zoll), 32 GB, Android) [Spanische Version] Schwarz/Silber', 8.158666666666667)] 

Bubbles ->  [('Nokia Asha 300 Sim Free Mobile Phone - Graphite', 8.158666666666667)] 

cranberrytwo ->  [('Samsung Galaxy Express I8730', 8.158666666666667)] 

Anand ->  [('OnePlus 3 (Graphite, 64 GB)', 8.158666666666667)] 

Rt, India ->  [('Nokia Lumia 520 (Cyan)', 8.158666666666667)] 

Guillaume ->  [('HTC One S 16GB Unlocked GSM Smartphone - International Version, No Warranty (Grey Blue)', 8.158666666666667)] 

perniciaro antonella patrizia ->  [('LG Nexus 5X Smartphone da 16 GB, Android 6.0, Display Full HD IPS da 5.2", 4G, Bianco [Italia]', 8.158666666666667)] 

davidopics2  ->  [('Samsung Galaxy S6 edge+ 32GB (Verizon)', 8.158666666666667)] 

Luka ->  [('Sony Ericsson Aino Handy (UMTS, Wifi, 8 MP, Dockingstation, inkl. 8GB Speicherkarte) luminous white', 8.158666666666667)] 

gangsta ->  [('Samsun

In [74]:
# Cross validation
svd_cv = cross_validate(svd,df2, measures=['RMSE'], cv=5, verbose=False)
svd_cv

{'test_rmse': array([2.5161215 , 2.63238777, 2.56680605, 2.68263155, 2.52114471]),
 'fit_time': (0.927168607711792,
  0.05576610565185547,
  0.056624412536621094,
  0.06502914428710938,
  0.07128763198852539),
 'test_time': (0.008000612258911133,
  0.003999471664428711,
  0.0039997100830078125,
  0.007999658584594727,
  0.005010843276977539)}

In [75]:
knn_i_cv = cross_validate(knn_i,df2, measures=['RMSE'], cv=5, verbose=False)
knn_i_cv

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([2.62758908, 2.59319793, 2.67159379, 2.58730064, 2.67219945]),
 'fit_time': (0.17049074172973633,
  0.1679222583770752,
  0.13541579246520996,
  0.14022397994995117,
  0.18000292778015137),
 'test_time': (0.021257400512695312,
  0.017987728118896484,
  0.012037038803100586,
  0.017772912979125977,
  0.015000343322753906)}

In [77]:
knn_u_cv = cross_validate(knn_u,df2, measures=['RMSE'], cv=5, verbose=False)
knn_u_cv

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([2.56327868, 2.54519131, 2.61733948, 2.69313957, 2.63527355]),
 'fit_time': (0.17435526847839355,
  0.20094919204711914,
  0.14297890663146973,
  0.1409289836883545,
  0.16904568672180176),
 'test_time': (0.006068706512451172,
  0.00500035285949707,
  0.006006002426147461,
  0.004998683929443359,
  0.005278825759887695)}

In [78]:
print(svd_cv['test_rmse'].mean())
print(knn_i_cv['test_rmse'].mean())
print(knn_u_cv['test_rmse'].mean())

2.583818316048105
2.63037617661006
2.610844519425876




There are wide variety of recommendation systems that can be used on case specific criteria which can improve the recommendation for different users like Demographic, Utility based, Knowledge based and Hybrid recommendation system.
